# Olivine Phase Loop
This notebook demonstrates calculation of the olivine liquid-solid phase loop under the assumption that both phases behave as ideal solutions.  
The workflow is:
- Use the **coder** module to generate endmember properties of both solutions; the only thermodynamic properties that are specified are the enthalpy and entropy of fusion
- Use the **coder** module to generate solid and liquid solution properties
- Import the generated code using the **model** module
- Use the **equilibrate** module to compute the liquid-solid phase loop
- Plot results

In [ ]:
import numpy as np
import scipy as sp
import sympy as sym
import matplotlib.pyplot as plt
from thermoengine import model, equilibrate, coder
%matplotlib inline

## Endmember properties
Write code into a working subdirectory

In [ ]:
working_dir = "working"
!mkdir -p {working_dir}
%cd {working_dir}

Model generation function

In [ ]:
def make_endmembers(module='none', name='none', formula='none', Hrefvalue=0.0, Srefvalue=0.0):
    mdl = coder.StdStateModel()
    T = mdl.get_symbol_for_t()
    GPr,Href,Sref = sym.symbols('GPr Href Sref')
    GPr = Href - T*Sref
    params = [('Href', 'J', Href), ('Sref', 'J/K', Sref)]
    mdl.add_expression_to_model(GPr, params)
    mdl.set_module_name(module)
    paramValues = {'Href':Hrefvalue, 'Sref':Srefvalue, 'T_r':298.15, 'P_r':1.0}
    mdl.create_code_module(phase=name, formula=formula, params=paramValues, 
                           module_type='calib', silent=True)

### Forsterite Solid

In [ ]:
make_endmembers(module='OlvSolid', name='Fo', formula='Mg(2)Si(1)O(4)', Hrefvalue=-100000.0, Srefvalue=0.0)
%cp OlvSolid.pyx endmembersolids.pyx

### Fayalite Solid

In [ ]:
make_endmembers(module='OlvSolid', name='Fa', formula='Fe(2)Si(1)O(4)', Hrefvalue=-100000.0, Srefvalue=0.0)
%cat OlvSolid.pyx >> endmembersolids.pyx

### Forsterite Liquid
Fusion temperature is 2163 K, entropy is 57.2 J/K

In [ ]:
make_endmembers(module='OlvLiquid', name='Fo', formula='Mg(2)Si(1)O(4)', Hrefvalue=-100000.0+57.2*2163.0, Srefvalue=57.2)
%cp OlvLiquid.pyx endmemberliquids.pyx

### Fayalite Liquid
Fusion temperature is 1490 K, entropy is 59.9 J/K

In [ ]:
make_endmembers(module='OlvLiquid', name='Fa', formula='Fe(2)Si(1)O(4)', Hrefvalue=-100000.0+59.9*1490.0, Srefvalue=59.9)
%cat OlvLiquid.pyx >> endmemberliquids.pyx

## Solution Properties
Model generation function

In [ ]:
def make_solution(module='none', name='none', endmembers=[]):
    c = 2
    mdl = coder.SimpleSolnModel(nc=c)
    n = mdl.n
    nT = mdl.nT
    X = n/nT
    T = mdl.get_symbol_for_t()
    mu = mdl.mu
    G_ss = (n.transpose()*mu)[0]
    S_config,R = sym.symbols('S_config R')
    S_config = 0
    for i in range(0,c):
        S_config += X[i]*sym.log(X[i])
    S_config *= -R*nT
    G_config = sym.simplify(-T*S_config)
    G = G_ss + G_config
    mdl.add_expression_to_model(G, [('dummy', 'none', sym.symbols('dummy'))])
    mdl.module = module
    mdl.formula_string = 'Mg[Mg]Fe[Fe]Si[Si]O4'
    mdl.conversion_string = ['[0]=[Mg]', '[1]=[Fe]']
    mdl.test_string = ['[0] > 0.0', '[1] > 0.0']
    mdl.create_code_module(phase=name, params={'dummy':0.0, 'T_r':298.15, 'P_r':1.0}, 
                           endmembers=endmembers, 
                           prefix="cy", module_type='calib', silent=True)

### Solid solution

In [ ]:
make_solution(module='OlvSolid', name='Olivine', endmembers=['Fo_OlvSolid', 'Fa_OlvSolid'])
%cat endmembersolids.pyx >> OlvSolid.pyx

### Liquid solution

In [ ]:
make_solution(module='OlvLiquid', name='Liquid', endmembers=['Fo_OlvLiquid', 'Fa_OlvLiquid'])
%cat endmemberliquids.pyx >> OlvLiquid.pyx

In [ ]:
import OlvSolid
import OlvLiquid
%cd ..

## Set up phase loop calculation

In [ ]:
modelDBsol = model.Database(database="CoderModule", calib='calib', 
                            phase_tuple=('OlvSolid', {
                                'Ol':['Olivine','solution'],
                                'Fo':['Fo','pure'],
                                'Fa':['Fa','pure']
                            }))
modelDBliq = model.Database(database="CoderModule", calib='calib', 
                            phase_tuple=('OlvLiquid', {
                                'Liq':['Liquid','solution'],
                                'Fo':['Fo','pure'],
                                'Fa':['Fa','pure']
                            }))

In [ ]:
olivine = modelDBsol.get_phase("Ol")
liquid = modelDBliq.get_phase("Liq")

In [ ]:
elm_sys = ['O','Mg','Si','Fe']
phs_sys = [liquid, olivine]

### Compute the loop

In [ ]:
xFoSol = [1.0]
xFoLiq = [1.0]
tC = [2163.0-273.15]
p = 1.0
for i in range(1,20):
    XFo = 1.0 - i*0.05
    XFa = 1.0 - XFo
    blk_cmp = np.array([4.0*(XFo+XFa), 2.0*XFo, XFo+XFa, 2.0*XFa])
    equil = equilibrate.Equilibrate(elm_sys, phs_sys)
    t = 2163.0*XFo + 1490.0*XFa
    state = equil.execute(t, p, bulk_comp=blk_cmp, debug=0, stats=True)
    state.print_state()
    tC.append(t-273.15)
    xFoSol.append(state.compositions(phase_name='Olivine', units='mole_frac')[0])
    xFoLiq.append(state.compositions(phase_name='Liquid', units='mole_frac')[0])
xFoSol.append(0.0)
xFoLiq.append(0.0)
tC.append(1490.0-273.15)

In [ ]:
plt.plot(xFoSol, tC, 'b-')
plt.plot(xFoLiq, tC, 'r-')
plt.ylabel('T °C')
plt.xlabel('Mole fraction')
plt.xlim(0.0, 1.0)